<a href="https://colab.research.google.com/github/Johnny-Ambulay/Taller/blob/main/DUPD_sesion3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://miro.medium.com/v2/resize:fit:1400/1*zomnCz8-VMgrSV8eLCx2tw.gif" width="200" alt="molecula"  />

$$\Large \textit{Cuadernos de Código de Python para Docentes Universitarios}$$

---
$$\large\textbf{Sesión 3: Inteligencia artificial para la consulta de textos | Parte I}$$

---
$$\textit{Base de datos de artículos científicos & Consultas en DataFrames}$$

<br>
<br>

Autor: __Jesus Alvarado-Huayhuaz__

# Instalaciones

In [1]:
%%capture
!pip install -U "transformers==4.40.0" --upgrade
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate

In [2]:
#@title Cargando LLAMA3
import transformers
import torch

model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 1er test

In [3]:
def get_completion_llama3(input):
  messages = [
    {"role"   : "system",
     "content": context1},
    {"role"   : "user",
     "content": """Eres un especialista en descubrimiento de farmacos y uso de open babel"""},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  return outputs[0]["generated_text"][len(prompt):]

In [4]:
context1 = "Indicame un codigo para convertir un dataframe que contiene moleculas en codigo SMILES en estructuras 3D optimizadas con MMFF94"

In [5]:
get_completion_llama3(context1)

"¡Haha! Me alegra poder ayudarte con eso. Aquí te dejo un ejemplo de código en Python que utiliza la biblioteca OpenBabel para convertir un dataframe que contiene moléculas en código SMILES a estructuras 3D optimizadas con MMFF94:\n```python\nimport pandas as pd\nfrom openbabel import pybel\nfrom openbabel.mmff94 import MMFF94\n\n# Carga el dataframe con las moléculas en código SMILES\ndf = pd.read_csv('molécules.csv')\n\n# Crea un objeto OpenBabel para trabajar con las moléculas\nob = pybel.readstring('smi', df['SMILES'].iloc[0])  # Lee la primera molécula del dataframe\n\n# Define la función para optimizar las moléculas con MMFF94\ndef optimize_mol(mol):\n    mmff94 = MMFF94()\n    mol.prepareMMFF94()\n    mmff94.optimize(mol)\n    return mol\n\n# Aplica la función para optimizar las moléculas en el dataframe\ndf['3D_Structure'] = df['SMILES'].apply(optimize_mol)\n\n# Guarda el dataframe con"

In [ ]:
get_completion_llama3(context1)

"¡Haha! Como especialista en descubrimiento de farmacos y uso de Open Babel, te puedo proporcionar un ejemplo de código en Python que convierte un dataframe que contiene moléculas en código SMILES a estructuras 3D optimizadas con MMFF94 utilizando Open Babel.\n\n Primero, debes instalar Open Babel y sus herramientas de Python, llamadas `ob` y `obconvert`. Puedes hacerlo mediante pip:\n```\npip install openbabel\n```\nUna vez instalado, puedes utilizar el siguiente código:\n```python\nimport pandas as pd\nimport openbabel\nfrom openbabel import pybel\n\n# Carga el dataframe con moléculas en código SMILES\ndf = pd.read_csv('molecules_smiles.csv')\n\n# Crea un objeto Open Babel para cada molécula en el dataframe\nmols = []\nfor index, row in df.iterrows():\n    smi = row['smiles']\n    mol = pybel.readstring('smi', smi)\n    mols.append(mol)\n\n# Optimiza las estructuras 3D con MMFF94\nfor i, mol in enumerate(mols):\n    mol.setkekcal()"

# Consultas

In [7]:
def consultas(input):
  messages = [
    {"role"   : "system",
     "content": context1},
    {"role"   : "user",
     "content": "Eres un especialista en física y en la enseñanza de la historia de la física, dime unicamente el nombre de quién es el principal cientifico que estudió: "},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  return outputs[0]["generated_text"][len(prompt):]

In [8]:
context1 = "Mecánica clásica"

In [9]:
consultas(context1)

'La mecánica clásica! El principal científico que estudió la mecánica clásica es Isaac Newton (1643-1727).'

## En una celda

In [11]:
import pandas as pd

data = {'Topic': ['Classical Mechanics', 'Thermodynamics', 'Electromagnetism', 'Optics', 'Modern Physics', 'Biophysics'], 'Subtopics': ['Kinematics, Dynamics, Rotational Motion, Oscillations and Waves, Gravitation', 'Temperature and Heat, First Law, Second Law, Third Law', 'Electrostatics, Electric Current, Magnetism, Electromagnetic Induction, Electromagnetic Waves', 'Geometric Optics, Physical Optics', 'Special Relativity, Quantum Physics, Nuclear Physics', 'Fluid Physics, Solid State Physics, Acoustics, Astrophysics']}
df = pd.DataFrame(data)

In [12]:
df

,Topic,Subtopics
0,Classical Mechanics,"Kinematics, Dynamics, Rotational Motion, Oscil..."
1,Thermodynamics,"Temperature and Heat, First Law, Second Law, T..."
2,Electromagnetism,"Electrostatics, Electric Current, Magnetism, E..."
3,Optics,"Geometric Optics, Physical Optics"
4,Modern Physics,"Special Relativity, Quantum Physics, Nuclear P..."
5,Biophysics,"Fluid Physics, Solid State Physics, Acoustics,..."


In [18]:
df["Topic"]

,Topic
0,Classical Mechanics
1,Thermodynamics
2,Electromagnetism
3,Optics
4,Modern Physics
5,Biophysics


In [19]:
df["Topic"] [4]

'Modern Physics'

In [16]:
context1 = df["Topic"][0]

In [17]:
context1 = df["Topic"][4]

In [20]:
consultas(context1)

'¡Claro! El principal científico que estudió la física moderna es Albert Einstein.'

## En múltiples celdas

In [21]:
df['Representante'] = None

for i in range(6):
  context1 = df["Topic"][i]
  response = consultas(context1)
  df['Representante'][i] = response

In [22]:
df

,Topic,Subtopics,Representante
0,Classical Mechanics,"Kinematics, Dynamics, Rotational Motion, Oscil...",Un clásico!\n\nEl principal científico que est...
1,Thermodynamics,"Temperature and Heat, First Law, Second Law, T...",La termodinámica! El principal científico que ...
2,Electromagnetism,"Electrostatics, Electric Current, Magnetism, E...",Michael Faraday
3,Optics,"Geometric Optics, Physical Optics",La óptica!
4,Modern Physics,"Special Relativity, Quantum Physics, Nuclear P...",Un tema interesante!\n\nEl principal científic...
5,Biophysics,"Fluid Physics, Solid State Physics, Acoustics,...",La biología y la física! Uno de los científico...


In [23]:
df["Representante"][0]

'Un clásico!\n\nEl principal científico que estudió la mecánica clásica es: Isaac Newton.'

In [ ]:
df["Representante"][1]

'Sí! El principal científico que estudió la termodinámica fue Sadi Carnot, un físico francés que publicó su obra "Reflexiones sobre la puissance motrice du feu" en 1824.'

In [24]:
df["Representante"][2]

'Michael Faraday'

In [26]:
df["Representante"][3]

'La óptica!'

In [27]:
df["Representante"][4]

'Un tema interesante!\n\nEl principal científico que estudió la física moderna es Albert Einstein (1879-1955).'

In [29]:
df["Representante"][5]

'La biología y la física! Uno de los científicos más importantes en este campo es Max Delbrück, un biólogo y físico alemán-estadounidense que trabajó en la aplicación de la física y la química para entender la biología.'

## score

In [35]:
def score(input):
  messages = [
    {"role"   : "system",
     "content": context1},
    {"role"   : "user",
     "content": "Eres un especialista en física y en la enseñanza de la historia de la física, puntúa del 1 al 5, donde 1 es lo peor y 5 es lo mejor, imprime solamente el número o puntuacion de: ¿qué aporte tuvo Sir Isaac Newton en esta area?: "},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  return outputs[0]["generated_text"][len(prompt):]

In [36]:
df['Score_1'] = None

for i in range(6):
  context1 = df["Topic"][i]
  response = score(context1)
  df['Score_1'][i] = response

In [37]:
df

,Topic,Subtopics,Representante,Score_1
0,Classical Mechanics,"Kinematics, Dynamics, Rotational Motion, Oscil...",Un clásico!\n\nEl principal científico que est...,5
1,Thermodynamics,"Temperature and Heat, First Law, Second Law, T...",La termodinámica! El principal científico que ...,5
2,Electromagnetism,"Electrostatics, Electric Current, Magnetism, E...",Michael Faraday,5
3,Optics,"Geometric Optics, Physical Optics",La óptica!,5
4,Modern Physics,"Special Relativity, Quantum Physics, Nuclear P...",Un tema interesante!\n\nEl principal científic...,5
5,Biophysics,"Fluid Physics, Solid State Physics, Acoustics,...",La biología y la física! Uno de los científico...,5


In [ ]:
df

,Topic,Subtopics,Representante,Score_1
0,Classical Mechanics,"Kinematics, Dynamics, Rotational Motion, Oscil...",Un placer responder a esta pregunta!\n\nSin du...,5
1,Thermodynamics,"Temperature and Heat, First Law, Second Law, T...",Un placer responder!\n\nSir Isaac Newton es co...,5
2,Electromagnetism,"Electrostatics, Electric Current, Magnetism, E...",Un honor responder a esta pregunta! Sir Isaac ...,4
3,Optics,"Geometric Optics, Physical Optics",¡Un placer hablar sobre el gran Sir Isaac Newt...,5
4,Modern Physics,"Special Relativity, Quantum Physics, Nuclear P...",Un placer responder!\n\nMe atrevo a calificar ...,4
5,Biophysics,"Fluid Physics, Solid State Physics, Acoustics,...",La biofísica! El principal científico que estu...,5


In [ ]:
# borrar alguna columna
#df = df.drop('columna',axis=1)

In [38]:
type(df['Score_1'][0])

str

In [39]:
df['Score_1'] = df['Score_1'].astype(int)

In [40]:
df_filtrado = df[df['Score_1'] >= 4]
df_filtrado

,Topic,Subtopics,Representante,Score_1
0,Classical Mechanics,"Kinematics, Dynamics, Rotational Motion, Oscil...",Un clásico!\n\nEl principal científico que est...,5
1,Thermodynamics,"Temperature and Heat, First Law, Second Law, T...",La termodinámica! El principal científico que ...,5
2,Electromagnetism,"Electrostatics, Electric Current, Magnetism, E...",Michael Faraday,5
3,Optics,"Geometric Optics, Physical Optics",La óptica!,5
4,Modern Physics,"Special Relativity, Quantum Physics, Nuclear P...",Un tema interesante!\n\nEl principal científic...,5
5,Biophysics,"Fluid Physics, Solid State Physics, Acoustics,...",La biología y la física! Uno de los científico...,5


# Actividad

Seleccione 5 articulos de su base de datos y realice la pregunta al Abstract:

¿Cuál es el principal descubrimiento o aporte de este artículos en su area de investigación?

In [ ]:
# Para seleccionar un rango de filas
filas_seleccionadas = df.iloc[2:5]
filas_seleccionadas

,Topic,Subtopics
2,Electromagnetism,"Electrostatics, Electric Current, Magnetism, E..."
3,Optics,"Geometric Optics, Physical Optics"
4,Modern Physics,"Special Relativity, Quantum Physics, Nuclear P..."


In [41]:
def consultas(input):
  messages = [
    {"role"   : "system",
     "content": context1},
    {"role"   : "user",
     "content": "Eres un especialista en nutrición y en bioestadistica, dime ¿cual es el principal hallazgo del artículo?: "},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  return outputs[0]["generated_text"][len(prompt):]

In [42]:
context1 = "Nutrición y salud mental"

In [43]:
consultas(context1)

'¡Hola!\n\nDespués de revisar el artículo sobre nutrición y salud mental, puedo decirte que el principal hallazgo es que la ingesta adecuada de nutrientes, especialmente los omega-3, vitaminas D y B, y minerales como el magnesio y el potasio, puede tener un impacto significativo en la salud mental.\n\nEn particular, el artículo destaca que una ingesta adecuada de omega-3, especialmente el ácido alfa-linoleico (ALA), puede reducir el riesgo de depresión y ansiedad. Además, la deficiencia en vitaminas D y B12 se ha relacionado con un mayor riesgo de depresión y trastornos del estado de ánimo.\n\nAdemás, el artículo también menciona que la ingesta de alimentos ricos en fibra, como frutas, verduras y granos integrales, puede tener beneficios para la salud mental, ya que puede ayudar a regular el tránsito intestinal y reducir el estrés.\n\nEn resumen, el principal hallazgo es que una dieta equilibrada y rica en nutrientes es fundamental para la salud mental y puede tener un impacto signific

# **Presentación**

In [44]:
ds = pd.read_csv("https://raw.githubusercontent.com/inefable12/DUPD_2024/main/scopus.csv")

In [45]:
ds

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,Cited by,DOI,Link,Abstract,Funding Details,Document Type,Publication Stage,Open Access,Source,EID
0,Lim C.M.; Lal S.K.; Isa N.M.; Omar A.R.; Choo ...,"Lim, Chie Min (58293538800); Lal, Sunil Kumar ...",58293538800; 7203074991; 58927564900; 72028640...,Betacyanins from red pitahaya (Hylocereus poly...,2024,Heliyon,10,12,e33049,NaN,...,0,10.1016/j.heliyon.2024.e33049,https://www.scopus.com/inward/record.uri?eid=2...,Seasonal influenza affects millions of lives w...,"Ministry of Higher Education, Malaysia, MOHE",Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85196294235
1,Hao J.; Zhang X.; Hu R.; Lu X.; Wang H.; Li Y....,"Hao, JinFang (59164972400); Zhang, Xiaoyan (56...",59164972400; 56033384800; 59167456200; 5916756...,Metabolomics combined with network pharmacolog...,2024,Journal of Translational Medicine,22,1,555,NaN,...,0,10.1186/s12967-024-05355-9,https://www.scopus.com/inward/record.uri?eid=2...,"Background: Astragaloside IV (AST-IV), as an e...",Mega Research and Development Projects of Lüli...,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85195625440
2,Kim S.-R.; Lee J.-M.; Kang H.J.; Ryou J.; Shim...,"Kim, Seong-Ryeol (57054781500); Lee, Jung-Min ...",57054781500; 59167499700; 59167429600; 1405490...,Antiviral activity of pimecrolimus against den...,2024,Scientific Reports,14,1,13303,NaN,...,0,10.1038/s41598-024-61127-x,https://www.scopus.com/inward/record.uri?eid=2...,Dengue virus (DENV) infection is a public heal...,"Korea National Institute of Health, KNIH, (202...",Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85195629120
3,Kulabaş N.; Lee J.-C.; Bingöl Özakpınar Ö.; Kü...,"Kulabaş, Necla (57190582389); Lee, Jin-Ching (...",57190582389; 58976849400; 39161700400; 6603485785,Synthesis and anti-HCV activity of novel 5-ary...,2024,Journal of Molecular Structure,1311,NaN,138407,NaN,...,0,10.1016/j.molstruc.2024.138407,https://www.scopus.com/inward/record.uri?eid=2...,Hepatitis C (HCV) is a viral infection that le...,"Marmara Üniversitesi, (SAG-C-DRP-050614-0227);...",Article,Final,NaN,Scopus,2-s2.0-85191714624
4,Sahoo S.; Purohit P.; Samantaray S.; Meher B.R.,"Sahoo, Sthitaprajna (57224752520); Purohit, Pr...",57224752520; 57221340577; 57474573300; 1401368...,Identification of Antiviral Phytocompounds as ...,2024,ChemistrySelect,9,12,e202400384,NaN,...,0,10.1002/slct.202400384,https://www.scopus.com/inward/record.uri?eid=2...,Dengue fever is one of the most dreaded viral ...,"Department of Higher Education, India; Berhamp...",Article,Final,NaN,Scopus,2-s2.0-85188171372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951,Malewicz B.; Jenkin H.M.; Borowski E.,"Malewicz, B. (6701786025); Jenkin, H.M. (70037...",6701786025; 7003772380; 7005468384,Selective cytotoxic effect of polyene macrolid...,1981,Drugs under Experimental and Clinical Research,7,3,NaN,313,...,2,NaN,https://www.scopus.com/inward/record.uri?eid=2...,Polyene macrolide antibiotics containing large...,NaN,Article,Final,NaN,Scopus,2-s2.0-0019796593
4952,Waterman S.H.; Monath T.P.,"Waterman, S.H. (35278558800); Monath, T.P. (70...",35278558800; 7006623935,Fluorescent antibody techniques applied to the...,1982,Acta Virologica,26,5,NaN,376,...,6,NaN,https://www.scopus.com/inward/record.uri?eid=2...,We evaluated an indirect fluorescent antibody ...,NaN,Article,Final,NaN,Scopus,2-s2.0-0020378139
4953,Huynh Do P.; Caumes E.; Bricaire F.,"Huynh Do, P. (49361366500); Caumes, E. (710241...",49361366500; 7102414982; 16451248800,Emerging and reemerging infectious diseases - ...,2000,Praxis,89,4,NaN,125,...,2,NaN,https://www.scopus.com/inward/record.uri?eid=2...,We have recently seen a worldwide explosion of...,NaN,Article,Final,NaN,Scopus,2-s2.0-0034687896
4954,Koff W.C.; Elm Jr. J.L.; Halstead S.B.,"Koff, Wayne C. (7006203853); Elm Jr., Joe L. (...",7006203853; 56906797100; 7005704809,

In [47]:
tabla = pd.read_csv("https://raw.githubusercontent.com/inefable12/DUPD_2024/main/scopus.csv")

In [46]:
ds.columns

Index(['Authors', 'Author full names', 'Author(s) ID', 'Title', 'Year',
       'Source title', 'Volume', 'Issue', 'Art. No.', 'Page start', 'Page end',
       'Page count', 'Cited by', 'DOI', 'Link', 'Abstract', 'Funding Details',
       'Document Type', 'Publication Stage', 'Open Access', 'Source', 'EID'],
      dtype='object')

In [49]:
ds["Abstract"]

,Abstract
0,Seasonal influenza affects millions of lives w...
1,"Background: Astragaloside IV (AST-IV), as an e..."
2,Dengue virus (DENV) infection is a public heal...
3,Hepatitis C (HCV) is a viral infection that le...
4,Dengue fever is one of the most dreaded viral ...
...,...
4951,Polyene macrolide antibiotics containing large...
4952,We evaluated an indirect fluorescent antibody ...
4953,We have recently seen a worldwide explosion of...
4954,The antiviral effects of ribavirin (1-β-d-ribo...


In [50]:
ds

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,Cited by,DOI,Link,Abstract,Funding Details,Document Type,Publication Stage,Open Access,Source,EID
0,Lim C.M.; Lal S.K.; Isa N.M.; Omar A.R.; Choo ...,"Lim, Chie Min (58293538800); Lal, Sunil Kumar ...",58293538800; 7203074991; 58927564900; 72028640...,Betacyanins from red pitahaya (Hylocereus poly...,2024,Heliyon,10,12,e33049,NaN,...,0,10.1016/j.heliyon.2024.e33049,https://www.scopus.com/inward/record.uri?eid=2...,Seasonal influenza affects millions of lives w...,"Ministry of Higher Education, Malaysia, MOHE",Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85196294235
1,Hao J.; Zhang X.; Hu R.; Lu X.; Wang H.; Li Y....,"Hao, JinFang (59164972400); Zhang, Xiaoyan (56...",59164972400; 56033384800; 59167456200; 5916756...,Metabolomics combined with network pharmacolog...,2024,Journal of Translational Medicine,22,1,555,NaN,...,0,10.1186/s12967-024-05355-9,https://www.scopus.com/inward/record.uri?eid=2...,"Background: Astragaloside IV (AST-IV), as an e...",Mega Research and Development Projects of Lüli...,Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85195625440
2,Kim S.-R.; Lee J.-M.; Kang H.J.; Ryou J.; Shim...,"Kim, Seong-Ryeol (57054781500); Lee, Jung-Min ...",57054781500; 59167499700; 59167429600; 1405490...,Antiviral activity of pimecrolimus against den...,2024,Scientific Reports,14,1,13303,NaN,...,0,10.1038/s41598-024-61127-x,https://www.scopus.com/inward/record.uri?eid=2...,Dengue virus (DENV) infection is a public heal...,"Korea National Institute of Health, KNIH, (202...",Article,Final,All Open Access; Gold Open Access,Scopus,2-s2.0-85195629120
3,Kulabaş N.; Lee J.-C.; Bingöl Özakpınar Ö.; Kü...,"Kulabaş, Necla (57190582389); Lee, Jin-Ching (...",57190582389; 58976849400; 39161700400; 6603485785,Synthesis and anti-HCV activity of novel 5-ary...,2024,Journal of Molecular Structure,1311,NaN,138407,NaN,...,0,10.1016/j.molstruc.2024.138407,https://www.scopus.com/inward/record.uri?eid=2...,Hepatitis C (HCV) is a viral infection that le...,"Marmara Üniversitesi, (SAG-C-DRP-050614-0227);...",Article,Final,NaN,Scopus,2-s2.0-85191714624
4,Sahoo S.; Purohit P.; Samantaray S.; Meher B.R.,"Sahoo, Sthitaprajna (57224752520); Purohit, Pr...",57224752520; 57221340577; 57474573300; 1401368...,Identification of Antiviral Phytocompounds as ...,2024,ChemistrySelect,9,12,e202400384,NaN,...,0,10.1002/slct.202400384,https://www.scopus.com/inward/record.uri?eid=2...,Dengue fever is one of the most dreaded viral ...,"Department of Higher Education, India; Berhamp...",Article,Final,NaN,Scopus,2-s2.0-85188171372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4951,Malewicz B.; Jenkin H.M.; Borowski E.,"Malewicz, B. (6701786025); Jenkin, H.M. (70037...",6701786025; 7003772380; 7005468384,Selective cytotoxic effect of polyene macrolid...,1981,Drugs under Experimental and Clinical Research,7,3,NaN,313,...,2,NaN,https://www.scopus.com/inward/record.uri?eid=2...,Polyene macrolide antibiotics containing large...,NaN,Article,Final,NaN,Scopus,2-s2.0-0019796593
4952,Waterman S.H.; Monath T.P.,"Waterman, S.H. (35278558800); Monath, T.P. (70...",35278558800; 7006623935,Fluorescent antibody techniques applied to the...,1982,Acta Virologica,26,5,NaN,376,...,6,NaN,https://www.scopus.com/inward/record.uri?eid=2...,We evaluated an indirect fluorescent antibody ...,NaN,Article,Final,NaN,Scopus,2-s2.0-0020378139
4953,Huynh Do P.; Caumes E.; Bricaire F.,"Huynh Do, P. (49361366500); Caumes, E. (710241...",49361366500; 7102414982; 16451248800,Emerging and reemerging infectious diseases - ...,2000,Praxis,89,4,NaN,125,...,2,NaN,https://www.scopus.com/inward/record.uri?eid=2...,We have recently seen a worldwide explosion of...,NaN,Article,Final,NaN,Scopus,2-s2.0-0034687896
4954,Koff W.C.; Elm Jr. J.L.; Halstead S.B.,"Koff, Wayne C. (7006203853); Elm Jr., Joe L. (...",7006203853; 56906797100; 7005704809,

In [51]:
filas_seleccionadas = ds.iloc[3:5]
filas_seleccionadas

,Authors,Author full names,Author(s) ID,Title,Year,Source title,Volume,Issue,Art. No.,Page start,...,Cited by,DOI,Link,Abstract,Funding Details,Document Type,Publication Stage,Open Access,Source,EID
3,Kulabaş N.; Lee J.-C.; Bingöl Özakpınar Ö.; Kü...,"Kulabaş, Necla (57190582389); Lee, Jin-Ching (...",57190582389; 58976849400; 39161700400; 6603485785,Synthesis and anti-HCV activity of novel 5-ary...,2024,Journal of Molecular Structure,1311,NaN,138407,NaN,...,0,10.1016/j.molstruc.2024.138407,https://www.scopus.com/inward/record.uri?eid=2...,Hepatitis C (HCV) is a viral infection that le...,"Marmara Üniversitesi, (SAG-C-DRP-050614-0227);...",Article,Final,NaN,Scopus,2-s2.0-85191714624
4,Sahoo S.; Purohit P.; Samantaray S.; Meher B.R.,"Sahoo, Sthitaprajna (57224752520); Purohit, Pr...",57224752520; 57221340577; 57474573300; 1401368...,Identification of Antiviral Phytocompounds as ...,2024,ChemistrySelect,9,12,e202400384,NaN,...,0,10.1002/slct.202400384,https://www.scopus.com/inward/record.uri?eid=2...,Dengue fever is one of the most dreaded viral ...,"Department of Higher Education, India; Berhamp...",Article,Final,NaN,Scopus,2-s2.0-85188171372


In [52]:
filas_seleccionadas.to_csv("mitabla.csv")

In [53]:
ls

mitabla.csv  sample_data/


In [ ]:
def consultas(input):
  messages = [
    {"role"   : "system",
     "content": context1},
    {"role"   : "user",
     "content": "Eres un especialista en dengue, simulaciones moleculares y en analisis de texto cientifico ¿?"},
  ]

  prompt = pipeline.tokenizer.apply_chat_template(
          messages,
          tokenize=False,
          add_generation_prompt=True
  )

  terminators = [
      pipeline.tokenizer.eos_token_id,
      pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = pipeline(
      prompt,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )

  return outputs[0]["generated_text"][len(prompt):]

In [ ]:
df['Analisis1'] = None

for i in range(3):
  context1 = df["Abstract"][i]
  response = consultas(context1)
  df['Analisis1'][i] = response

In [ ]:
df['Analisis1']

